In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
# use process bar tool
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-final-dataset/dataset/test/adults/20.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/6.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/76.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/71.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/5.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/8.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/84.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/85.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/67.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/82.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/30.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/10.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/0.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/62.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/61.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/73.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/60.jpg
/kaggle/input/ml-f

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [3]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/test', transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [4]:
# Load pre-trained EfficientNet model
model = models.efficientnet_b0(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False

# Modify the final layer to match the number of classes in the final project
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)  # Binary classification (2 classes)

# Transfer model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 108MB/s] 


In [5]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Use tqdm to create a progress bar for the data loader
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch"):
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch 1/10: 100%|██████████| 18/18 [00:05<00:00,  3.31batch/s]


Epoch [1/10], Loss: 0.6586


Epoch 2/10: 100%|██████████| 18/18 [00:03<00:00,  6.00batch/s]


Epoch [2/10], Loss: 0.5308


Epoch 3/10: 100%|██████████| 18/18 [00:02<00:00,  6.11batch/s]


...

Epoch [8/10], Loss: 0.3552


Epoch 9/10: 100%|██████████| 18/18 [00:02<00:00,  6.28batch/s]


Epoch [9/10], Loss: 0.3721


Epoch 10/10: 100%|██████████| 18/18 [00:02<00:00,  6.30batch/s]

Epoch [10/10], Loss: 0.3700


In [6]:
# Evaluate the model on the test dataset
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 75.83%


In [7]:
# Evaluate the model on the train dataset
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Train Accuracy: {100 * correct / total:.2f}%')

Train Accuracy: 90.36%


In [8]:
torch.save(model, '2024_06_15_trial.pt')

In [9]:
print("Model Layers:")
orig_model = models.efficientnet_b0(pretrained=True)
for layer in orig_model.children():
    print(layer)

Model Layers:
Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(32, 16, kernel_size=(1, 1), s